In [ ]:
# Test anomaly detection performance

In [1]:
import os
import random
import pickle
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

from keras.optimizers import Adam
from models import AE
# from models import USAD
from models import USAD_keras
from utils import *

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
seed_everything(42)

In [62]:
# make dataset(building == 1)
df = pd.read_csv('./datasets/lead1.0-small.csv').dropna().set_index('timestamp')
df = df[df['building_id']==1].drop('building_id', axis=1)


# split(train | val | test) = (6:2:2)
size = len(df)

train = df[:int(size*.6)]
train = train[train['anomaly']==0]
val = df[int(size*.6):int(size*.8)]
test = df[int(size*.8):]

train = create_seq(train, seq_len=24)
val = create_seq(val, seq_len=24)
test = create_seq(test, seq_len=24)

print(f'Train: {train.shape}')
print(f'Val: {val.shape}')
print(f'Test: {test.shape}')

Train: (3142, 24, 2)
Val: (1041, 24, 2)
Test: (1041, 24, 2)


In [63]:
train_X = train[:, :, 0:1]
train_y = train[:, :, 1:]

val_X = val[:, :, 0:1]
val_y = val[:, :, 1:]

test_X = test[:, :, 0:1]
test_y = test[:, :, 1:]

In [64]:
# Min-Max norm
min_val, max_val = np.min(train_X), np.max(train_X)
train_norm = minmax(train_X, min_val, max_val)
val_norm = minmax(val_X, min_val, max_val)
test_norm = minmax(test_X, min_val, max_val)

# make tf dataset
train_dataset = make_dataset(train_norm, train_norm)
val_dataset = make_dataset(val_norm, val_norm)
test_dataset = make_dataset(test_norm, test_norm)

# AutoEncoder(AE)
- LSTM-AE
- GRU-AE
- TCN-AE

In [65]:
# LSTM-AE
lstmae = AE(mode='LSTM')
# GRU-AE
gruae = AE(mode='GRU')
# TCN-AE
tcnae = AE(z_dim=32, hidden_dim=32, dilations=[1, 2, 4], mode='TCN')

input_shape = (None, train.shape[1], train.shape[2]-1)
lstmae.build(input_shape)
gruae.build(input_shape)
tcnae.build(input_shape)

lstmae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')
gruae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')
tcnae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')

lstm_time = TimeHistory()
gru_time = TimeHistory()
tcn_time = TimeHistory()

In [66]:
print(lstmae.summary(), gruae.summary())

Model: "ae_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_3 (Encoder)         multiple                  16896     
                                                                 
 decoder_3 (Decoder)         multiple                  115457    
                                                                 
Total params: 132,353
Trainable params: 132,353
Non-trainable params: 0
_________________________________________________________________
Model: "ae_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_4 (Encoder)         multiple                  12864     
                                                                 
 decoder_4 (Decoder)         multiple                  91137     
                                                                 
Total params: 104,001
Trainable params: 10

In [67]:
print(tcnae.summary())

Model: "ae_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_5 (Encoder)         multiple                  35104     
                                                                 
 decoder_5 (Decoder)         multiple                  16545     
                                                                 
Total params: 51,649
Trainable params: 51,649
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
epochs = 50
batch_size = 64

lstmaehist = lstmae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[lstm_time])
print(f'LSTM-AE train time: {sum(lstm_time.times):.4f}')

Epoch 1/50
50/50 [==============================] - 10s 24ms/step - loss: 0.0535 - mae: 0.1609 - val_loss: 0.0279 - val_mae: 0.1242
Epoch 2/50
50/50 [==============================] - 0s 8ms/step - loss: 0.0118 - mae: 0.0763 - val_loss: 0.0249 - val_mae: 0.1182
Epoch 3/50
50/50 [==============================] - 0s 8ms/step - loss: 0.0088 - mae: 0.0667 - val_loss: 0.0217 - val_mae: 0.1104
Epoch 4/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0075 - mae: 0.0629 - val_loss: 0.0196 - val_mae: 0.1031
Epoch 5/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0067 - mae: 0.0595 - val_loss: 0.0177 - val_mae: 0.0975
Epoch 6/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0061 - mae: 0.0561 - val_loss: 0.0163 - val_mae: 0.0939
Epoch 7/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0056 - mae: 0.0540 - val_loss: 0.0155 - val_mae: 0.0932
Epoch 8/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0054 - mae: 0.

In [69]:
gruaehist = gruae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[gru_time])
print(f'GRU-AE train time: {sum(gru_time.times):.4f}')

Epoch 1/50
50/50 [==============================] - 3s 22ms/step - loss: 0.0450 - mae: 0.1438 - val_loss: 0.0288 - val_mae: 0.1247
Epoch 2/50
50/50 [==============================] - 1s 10ms/step - loss: 0.0098 - mae: 0.0678 - val_loss: 0.0282 - val_mae: 0.1234
Epoch 3/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0095 - mae: 0.0681 - val_loss: 0.0267 - val_mae: 0.1204
Epoch 4/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0090 - mae: 0.0662 - val_loss: 0.0255 - val_mae: 0.1180
Epoch 5/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0085 - mae: 0.0644 - val_loss: 0.0242 - val_mae: 0.1154
Epoch 6/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0081 - mae: 0.0625 - val_loss: 0.0232 - val_mae: 0.1130
Epoch 7/50
50/50 [==============================] - 0s 10ms/step - loss: 0.0077 - mae: 0.0611 - val_loss: 0.0220 - val_mae: 0.1099
Epoch 8/50
50/50 [==============================] - 0s 10ms/step - loss: 0.0073 - mae: 

In [70]:
tcnaehist = tcnae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[tcn_time])
print(f'TCN-AE train time: {sum(tcn_time.times):.4f}')

Epoch 1/50
50/50 [==============================] - 5s 13ms/step - loss: 0.0928 - mae: 0.2094 - val_loss: 0.0317 - val_mae: 0.1352
Epoch 2/50
50/50 [==============================] - 0s 8ms/step - loss: 0.0092 - mae: 0.0675 - val_loss: 0.0253 - val_mae: 0.1194
Epoch 3/50
50/50 [==============================] - 0s 8ms/step - loss: 0.0082 - mae: 0.0649 - val_loss: 0.0219 - val_mae: 0.1130
Epoch 4/50
50/50 [==============================] - 0s 8ms/step - loss: 0.0075 - mae: 0.0631 - val_loss: 0.0190 - val_mae: 0.1059
Epoch 5/50
50/50 [==============================] - 0s 8ms/step - loss: 0.0061 - mae: 0.0569 - val_loss: 0.0170 - val_mae: 0.0981
Epoch 6/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0058 - mae: 0.0554 - val_loss: 0.0156 - val_mae: 0.0945
Epoch 7/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0052 - mae: 0.0520 - val_loss: 0.0140 - val_mae: 0.0870
Epoch 8/50
50/50 [==============================] - 0s 9ms/step - loss: 0.0046 - mae: 0.0

In [71]:
path = './save/LEAD_1/'
check_path(path)
lstmae.save_weights(path + 'LSTMAE.h5')
gruae.save_weights(path + 'GRUAE.h5')
tcnae.save_weights(path + 'TCNAE.h5')

In [72]:
save_hist(lstmaehist, lstm_time, path + 'LSTMAEhist.pkl')
save_hist(gruaehist, gru_time, path + 'GRUAEhist.pkl')
save_hist(tcnaehist, tcn_time, path + 'TCNAEhist.pkl')